In [1]:
import pandas as pd
import os
import sys
from glob import glob
import numpy as np
# from sklearn.decomposition import PCA  # , KernelPCA  # runs out of ram
# import umap  # runs out of ram
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import gc
from IPython.display import clear_output

main_dir = os.path.abspath(os.pardir)
sys.path.insert(0, main_dir)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100
pd.options.display.precision = 8
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
data_path = os.path.join(main_dir, "data")+os.sep
interim_path = os.path.join(data_path, "interim")+os.sep
csv_files = glob(interim_path+"*.csv")
ml_data_fname = "ml_data.gzip"
for i, file in enumerate(csv_files):
    if file.rsplit('\\', 1)[-1] == ml_data_fname:
        csv_files.remove(file)

# cmp_n_max = 20
# exp_var_rto = 0.03

In [14]:
df1 = pd.read_csv(interim_path+"cl.csv", low_memory=False, usecols=["filename", "class"])
train_ind = df1["filename"][~df1["filename"].str.startswith("tst_")].index

df1["rot_id"] = pd.to_numeric(df1["filename"].str.extract("_(\d+).jpg", expand=False).rename("rot_id"), downcast="unsigned")
df1 = df1[(df1["rot_id"].isin([0]))]
df1["flip"] = df1["filename"].str.extract("_(flip)_", expand=False).rename("flip").fillna("non_flip").astype("category")
df1 = df1[(df1["flip"]=="non_flip")]

df1 = df1.drop(columns=["rot_id", "flip"])

df1["type"] = df1["filename"].str.extract("^(trn|vld|tst)_", expand=False).rename("type").astype("category")
df1["add"] = df1["filename"].str.extract("_(add)_", expand=False).rename("add").fillna("non_add").astype("category")
df1["img_id"] = pd.to_numeric(df1["filename"].str.extract("_(\d+)_", expand=False).rename("img_id"), downcast="unsigned")

# df1["grp_key"] = (df1["type"].astype(str)+"_"+df1["class"].astype(str)+"_"+df1["add"].astype(str)+"_"+df1["img_id"].astype(str)).astype("category")

# img_grps = df1[["type", "add", "flip", "img_id","grp_key","rot_id"]].groupby("grp_key", sort=False, as_index=False).apply(
#     lambda x: x.sample(frac=1, random_state=123)).reset_index()
# df1 = pd.concat([df1, pd.to_numeric(pd.Series(img_grps["level_0"], index=img_grps["level_1"]).rename("img_grp"), downcast="unsigned")], axis=1)

# pca_redcr = dict()

object_vnames = df1.select_dtypes("O").columns
if len(object_vnames) > 0:
    df1[object_vnames] = df1[object_vnames].astype("category")

print(df1.shape)
df1.head()

(10239, 5)


,filename,class,type,add,img_id
0,trn_t1_1131_0.jpg,t1,trn,non_add,1131
90,trn_t1_1220_0.jpg,t1,trn,non_add,1220
180,trn_t1_1230_0.jpg,t1,trn,non_add,1230
270,trn_t1_1346_0.jpg,t1,trn,non_add,1346
360,trn_t1_245_0.jpg,t1,trn,non_add,245


In [15]:
print(df1.shape)

(10239, 5)


In [26]:
def safe_downcast(series_in, num_type):
    series_out = pd.to_numeric(series_in, downcast=num_type)
    if not np.equal(series_in, series_out).all():
        series_out = series_in
    return series_out

In [27]:
for i, file in enumerate(csv_files):
    prfx_str = file.rsplit('\\', 1)[-1].replace(".csv","")+"_"
    print("Progress", round(((i+1)/(len(csv_files)+1))*100, 2), "% file "+prfx_str+".csv", end="\r")
    df_temp = pd.read_csv(file, low_memory=False).drop(columns=["filename", "class"]).loc[df1.index]
    num_cols = df_temp.select_dtypes(np.number).columns
    drop_cols = num_cols[df_temp[num_cols].sum(axis=0) == 0]
    df_temp = df_temp.drop(columns=drop_cols)
#     num_cols = df_temp.select_dtypes(np.number).columns
    
#     # linear reduction
#     if len(num_cols) > cmp_n_max:
#         pca_cmpnts = cmp_n_max
#     else:
#         pca_cmpnts = len(num_cols)
#     pca_redcr[prfx_str] = PCA(n_components=pca_cmpnts, copy=False, random_state=123)
#     pca_redcr[prfx_str].fit(df_temp.loc[train_ind, num_cols])
#     train_data = pca_redcr[prfx_str].transform(df_temp[num_cols])
#     n_cmpnts = np.where(pca_redcr[prfx_str].explained_variance_ratio_ >= exp_var_rto)[0]
#     xplnt_var = pca_redcr[prfx_str].explained_variance_ratio_[:n_cmpnts[-1]+1].sum().round(2)
#     print(str(n_cmpnts[-1]+1)+" components explain "+str(xplnt_var*100)+"% variance.")
#     display(pd.DataFrame(pca_redcr[prfx_str].explained_variance_ratio_).T)
#     train_data = pd.DataFrame(train_data, dtype=np.float32)[n_cmpnts]
    
#     if len(n_cmpnts) > 1:
#         plt.figure(figsize=(10, 7))
#         plt.scatter(train_data[0], train_data[1],
#                     c=df1["class"].replace(["t1", "t2", "t3", np.nan], [1, 2, 3, 0]),
#                     edgecolor='none', alpha=0.5, cmap=cm.get_cmap("Spectral"))
#         plt.title(prfx_str)
#         plt.xlabel('component 1')
#         plt.ylabel('component 2')
#         plt.colorbar()
#         plt.show()
    
#     train_data = train_data.add_prefix(prfx_str)
#     df_temp = df_temp.drop(columns=num_cols)
#     df_temp = pd.concat([df_temp, train_data], axis=1)

    int_vnames = df_temp.select_dtypes("integer").columns
    float_vnames = df_temp.select_dtypes("floating").columns
    
    if len(int_vnames) > 0:
        df_temp[int_vnames] = df_temp[int_vnames].apply(safe_downcast, num_type="unsigned")
    if len(float_vnames) > 0:
        df_temp[float_vnames] = df_temp[float_vnames].apply(safe_downcast, num_type="float")
    
    df1 = pd.concat([df1, df_temp], axis=1)
    
    del df_temp  # , train_data
    gc.collect()
    clear_output(wait=True)
    
print(df1.shape)
display(df1.head())

(10239, 2655)


,filename,class,type,add,img_id,'Auto Color Correlogram0','Auto Color Correlogram1','Auto Color Correlogram2','Auto Color Correlogram3','Auto Color Correlogram48','Auto Color Correlogram49','Auto Color Correlogram50','Auto Color Correlogram51','Auto Color Correlogram52','Auto Color Correlogram53','Auto Color Correlogram54','Auto Color Correlogram55','Auto Color Correlogram56','Auto Color Correlogram57','Auto Color Correlogram58','Auto Color Correlogram59','Auto Color Correlogram60','Auto Color Correlogram61','Auto Color Correlogram62','Auto Color Correlogram63','Spatial Pyramid of Local Binary Patterns0','Spatial Pyramid of Local Binary Patterns1','Spatial Pyramid of Local Binary Patterns2','Spatial Pyramid of Local Binary Patterns3','Spatial Pyramid of Local Binary Patterns4','Spatial Pyramid of Local Binary Patterns5','Spatial Pyramid of Local Binary Patterns6','Spatial Pyramid of Local Binary Patterns7','Spatial Pyramid of Local Binary Patterns8','Spatial Pyramid of Local Binary Patterns9','Spatial Pyramid of Local Binary Patterns10','Spatial Pyramid of Local Binary Patterns11','Spatial Pyramid of Local Binary Patterns12','Spatial Pyramid of Local Binary Patterns13','Spatial Pyramid of Local Binary Patterns14','Spatial Pyramid of Local Binary Patterns15','Spatial Pyramid of Local Binary Patterns16','Spatial Pyramid of Local Binary Patterns17','Spatial Pyramid of Local Binary Patterns18','Spatial Pyramid of Local Binary Patterns19','Spatial Pyramid of Local Binary Patterns20','Spatial Pyramid of Local Binary Patterns21','Spatial Pyramid of Local Binary Patterns22','Spatial Pyramid of Local Binary Patterns23','Spatial Pyramid of Local Binary Patterns24',...,PHOG580,PHOG581,PHOG582,PHOG583,PHOG584,PHOG585,PHOG586,PHOG587,PHOG588,PHOG589,PHOG590,PHOG591,PHOG592,PHOG593,PHOG594,PHOG595,PHOG596,PHOG597,PHOG598,PHOG599,PHOG600,PHOG601,PHOG602,PHOG603,PHOG604,PHOG605,PHOG606,PHOG607,PHOG608,PHOG609,PHOG610,PHOG611,PHOG612,PHOG613,PHOG614,PHOG615,PHOG616,PHOG617,PHOG618,PHOG619,PHOG620,PHOG621,PHOG622,PHOG623,PHOG624,PHOG625,PHOG626,PHOG627,PHOG628,PHOG629
0,trn_t1_1131_0.jpg,t1,trn,non_add,1131,10,8,7,6,15,14,13,13,13,12,11,10,13,12,11,11,14,14,13,13,1,1,2,0,7,0,0,0,15,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,...,10,9,8,11,10,11,11,12,11,14,13,14,15,14,12,12,10,9,10,7,9,9,8,9,7,10,8,9,9,9,9,9,11,12,11,15,12,12,14,11,13,10,9,10,10,10,10,11,9,7
90,trn_t1_1220_0.jpg,t1,trn,non_add,1220,12,11,10,9,15,15,15,14,15,14,14,13,15,14,13,12,15,15,14,14,0,0,0,0,2,0,0,0,15,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,...,6,7,10,12,11,15,10,8,6,7,9,6,4,4,4,2,1,1,1,0,0,1,0,1,3,4,3,5,4,4,5,1,2,4,3,4,4,6,6,5,3,5,5,14,11,6,3,2,1,2
180,trn_t1_1230_0.jpg,t1,trn,non_add,1230,11,9,7,6,15,15,15,14,15,14,14,14,14,14,13,13,15,14,14,14,0,0,1,0,5,0,0,0,15,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,...,11,9,10,7,7,7,5,4,2,4,3,3,5,5,3,6,4,2,6,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
270,trn_t1_1346_0.jpg,t1,trn,non_add,1346,2,1,1,1,15,14,14,14,15,14,13,13,14,13,12,12,15,14,14,14,2,3,1,0,4,0,0,0,15,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,...,1,2,4,7,11,14,9,5,4,2,1,0,0,0,0,1,1,0,0,0,10,10,10,9,9,8,6,7,7,8,8,9,11,11,11,15,13,13,13,13,12,10,11,11,9,11,10,9,10,10
360,trn_t1_245_0.jpg,t1,trn,non_add,245,10,9,8,7,14,13,12,11,12,11,9,9,13,12,11,10,15,14,14,13,3,5,5,1,10,0,1,1,15,0,0,0,1,0,0,1,11,0,0,0,0,1,0,0,0,...,12,14,13,15,14,13,11,10,10,7,7,7,6,5,4,5,4,4,4,3,3,3,3,3,3,4,4,5,6,8,9,9,10,12,13,15,11,10,8,8,8,8,7,5,6,4,4,3,3,3


In [35]:
df1 = df1.loc[:,~df1.columns.duplicated()]
df1.shape

(10239, 1743)

## File saving

In [36]:
train_ind = df1["class"][df1["class"].notna()].index
test_ind = df1["class"][df1["class"].isna()].index

df2 = df1.loc[train_ind].reset_index(drop=True).append(df1.loc[test_ind], ignore_index=True)

In [37]:
# df2.dtypes

In [38]:
df2.to_parquet(interim_path+ml_data_fname, engine="fastparquet", compression="gzip", index=False)

In [25]:
# from datetime import datetime
# START_TIME = datetime.now()

# # one of the following lines is uncommented before execution
# s = [np.float64(1), np.float32(1), np.float16(1), 1.0]

# for j, k in enumerate(s):
#     for i in range(10000000):
#         s[j] = (s[j] + 8) * s[j] % 2399232

#     print(s[j])
#     print('Runtime:', datetime.now() - START_TIME)
    

# 1340409.0
# Runtime: 0:00:11.860372
# 1340409.0
# Runtime: 0:00:23.970025
# 1340409.0
# Runtime: 0:00:35.694635
# 1340409.0
# Runtime: 0:00:39.258874

In [59]:
# def umap_trans(n_neighbors=15, min_dist=0.1, n_components=2, plot=False):
#     reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="euclidean")
#     embedding = reducer.fit_transform(train_data[num_vnames])
#     print("Embedding shape", embedding.shape)
    
#     if plot:
#         sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
#         plt.scatter(embedding[:, 0], embedding[:, 1], c=[sns.color_palette()[x] for x in train_data["class"].astype(int)])
#         plt.gca().set_aspect('equal', 'datalim')
#         plt.title('UMAP projection of the dataset', fontsize=24)
#     return reducer

In [60]:
# umap_trans(5, 0.1, 2, plot=True)

In [61]:
# max_feat = 100
# umap_reducer = dict()
# for vname_type in num_bd_vnames.keys():
#     if len(num_bd_vnames[vname_type]) > max_feat//len(num_bd_vnames.keys()):
#         print(vname_type, end=" ")
#         umap_reducer[vname_type] = umap_trans(5, 0.1, int(max_feat//len(num_bd_vnames.keys())))

In [62]:
# def apply_umap(data, num_vnames=num_bd_vnames, return_reducer=False):
#     y = data["class"].copy()
#     X = pd.DataFrame()
#     for vname_type in num_bd_vnames.keys():
#         if vname_type in umap_reducer.keys():
#             umap_reducer[vname_type] = umap_trans(5, 0.1, int(max_feat//len(num_bd_vnames.keys())))
#             X = pd.concat(
#                 [X, pd.DataFrame(umap_reducer[vname_type].transform(data[num_bd_vnames[vname_type]]), index=data.index).add_prefix(vname_type)], axis=1)
#         else:
#             X = pd.concat([X, data[num_bd_vnames[vname_type]]], axis=1)
#     if return_reducer:
#         return X, y, umap_reducer
#     else:
#         return X, y

In [63]:
# X_train, y_train, umap_reducer = apply_umap(train_data, return_reducer=True)
# X_test, y_test = apply_umap(test_data)